# [1934. Confirmation Rate](https://leetcode.com/problems/confirmation-rate/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Signups
<pre>+----------------+----------+
| Column Name    | Type     |
+----------------+----------+
| user_id        | int      |
| time_stamp     | datetime |
+----------------+----------+</pre>
user_id is the column of unique values for this table.
Each row contains information about the signup time for the user with ID user_id.
 

Table: Confirmations

<pre>+----------------+----------+
| Column Name    | Type     |
+----------------+----------+
| user_id        | int      |
| time_stamp     | datetime |
| action         | ENUM     |
+----------------+----------+</pre>
(user_id, time_stamp) is the primary key (combination of columns with unique values) for this table.
user_id is a foreign key (reference column) to the Signups table.
action is an ENUM (category) of the type ('confirmed', 'timeout')
Each row of this table indicates that the user with ID user_id requested a confirmation message at time_stamp and that confirmation message was either confirmed ('confirmed') or expired without confirming ('timeout').
 

The confirmation rate of a user is the number of 'confirmed' messages divided by the total number of requested confirmation messages. The confirmation rate of a user that did not request any confirmation messages is 0. Round the confirmation rate to two decimal places.

Write a solution to find the confirmation rate of each user.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Signups table:
<pre>+---------+---------------------+
| user_id | time_stamp          |
+---------+---------------------+
| 3       | 2020-03-21 10:16:13 |
| 7       | 2020-01-04 13:57:59 |
| 2       | 2020-07-29 23:09:44 |
| 6       | 2020-12-09 10:39:37 |
+---------+---------------------+</pre>
Confirmations table:
<pre>+---------+---------------------+-----------+
| user_id | time_stamp          | action    |
+---------+---------------------+-----------+
| 3       | 2021-01-06 03:30:46 | timeout   |
| 3       | 2021-07-14 14:00:00 | timeout   |
| 7       | 2021-06-12 11:57:29 | confirmed |
| 7       | 2021-06-13 12:58:28 | confirmed |
| 7       | 2021-06-14 13:59:27 | confirmed |
| 2       | 2021-01-22 00:00:00 | confirmed |
| 2       | 2021-02-28 23:59:59 | timeout   |
+---------+---------------------+-----------+</pre>
Output: 
<pre>+---------+-------------------+
| user_id | confirmation_rate |
+---------+-------------------+
| 6       | 0.00              |
| 3       | 0.00              |
| 7       | 1.00              |
| 2       | 0.50              |
+---------+-------------------+</pre>
Explanation: 
User 6 did not request any confirmation messages. The confirmation rate is 0.
User 3 made 2 requests and both timed out. The confirmation rate is 0.
User 7 made 3 requests and all were confirmed. The confirmation rate is 1.
User 2 made 2 requests where one was confirmed and the other timed out. The confirmation rate is 1 / 2 = 0.5.

In [1]:
# Pandas schema

import pandas as pd

data = [[3, '2020-03-21 10:16:13'], [7, '2020-01-04 13:57:59'], [2, '2020-07-29 23:09:44'], [6, '2020-12-09 10:39:37']]
signups = pd.DataFrame(data, columns=['user_id', 'time_stamp']).astype(
    {'user_id': 'Int64', 'time_stamp': 'datetime64[ns]'})
data = [[3, '2021-01-06 03:30:46', 'timeout'], [3, '2021-07-14 14:00:00', 'timeout'],
        [7, '2021-06-12 11:57:29', 'confirmed'], [7, '2021-06-13 12:58:28', 'confirmed'],
        [7, '2021-06-14 13:59:27', 'confirmed'], [2, '2021-01-22 00:00:00', 'confirmed'],
        [2, '2021-02-28 23:59:59', 'timeout']]
confirmations = pd.DataFrame(data, columns=['user_id', 'time_stamp', 'action']).astype(
    {'user_id': 'Int64', 'time_stamp': 'datetime64[ns]', 'action': 'object'})

In [2]:
# to pyspark schema

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

signups_df = spark.createDataFrame(signups)
confirmations_df = spark.createDataFrame(confirmations)

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/05 12:07:39 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/05 12:07:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/05 12:07:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
signups_df.show()

+-------+-------------------+
|user_id|         time_stamp|
+-------+-------------------+
|      3|2020-03-21 10:16:13|
|      7|2020-01-04 13:57:59|
|      2|2020-07-29 23:09:44|
|      6|2020-12-09 10:39:37|
+-------+-------------------+


In [4]:
confirmations_df.show()

+-------+-------------------+---------+
|user_id|         time_stamp|   action|
+-------+-------------------+---------+
|      3|2021-01-06 03:30:46|  timeout|
|      3|2021-07-14 14:00:00|  timeout|
|      7|2021-06-12 11:57:29|confirmed|
|      7|2021-06-13 12:58:28|confirmed|
|      7|2021-06-14 13:59:27|confirmed|
|      2|2021-01-22 00:00:00|confirmed|
|      2|2021-02-28 23:59:59|  timeout|
+-------+-------------------+---------+


In [5]:
# in Spark Dataframe
import pyspark.sql.functions as F

signups_df.join(confirmations_df, on=['user_id'], how='left').select('user_id', 'action') \
    .groupBy('user_id') \
    .agg(F.round(F.avg(F.when(F.col('action') == 'confirmed', 1).otherwise(0)), 2).alias('confirmation_rate')) \
    .show()

+-------+-----------------+
|user_id|confirmation_rate|
+-------+-----------------+
|      3|              0.0|
|      7|              1.0|
|      2|              0.5|
|      6|              0.0|
+-------+-----------------+


In [6]:
# in Spark SQL
signups_df.createOrReplaceTempView('signups')
confirmations_df.createOrReplaceTempView('confirmations')

spark.sql('''
select signups.user_id, round(avg(case when action='confirmed' then 1 else 0 end),2) as confirmation_rate 
from signups
left join confirmations
on signups.user_id = confirmations.user_id
group by signups.user_id
''').show()

+-------+-----------------+
|user_id|confirmation_rate|
+-------+-----------------+
|      3|              0.0|
|      7|              1.0|
|      2|              0.5|
|      6|              0.0|
+-------+-----------------+


In [7]:
spark.stop()